# SCAL3 demo

This demo uses the [Rust prototype](src/README.md) via its foreign function interface (FFI).

In [3]:
%pip install cbor2 fastecdsa

from ctypes import cdll, memmove, string_at
from cbor2 import dumps, loads, load
from fastecdsa import keys, curve, ecdsa
from fastecdsa.encoding.sec1 import SEC1Encoder
from fastecdsa.encoding.der import DEREncoder
from fastecdsa.encoding.util import int_to_bytes
import secrets
import hmac
import hashlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
lib = cdll.LoadLibrary("target/release/libscal3.so")
size = lib.scal3_buffer_size()
buf = lib.scal3_buffer_allocate()

def write_request(request):
    data = dumps(request)
    memmove(buf, data, len(data))
def read_response():
    return loads(string_at(buf, size))
def ecdh(sk, pk):
    pk = SEC1Encoder().decode_public_key(pk, curve.P256)
    return SEC1Encoder().encode_public_key(sk * pk, compressed=False)[1:33]

# Setup

sk_provider, pk_provider = keys.gen_keypair(curve.P256)
pk_provider = SEC1Encoder.encode_public_key(pk_provider)
k_provider = secrets.token_bytes(32)

# Enrolment

sk_device, pk_device = keys.gen_keypair(curve.P256)
pk_device = SEC1Encoder.encode_public_key(pk_device)
k_subscriber = secrets.token_bytes(32)

subscriber_state = {
    'mask': hmac.digest(k_subscriber, b"123456", hashlib.sha256),
    'randomness': secrets.token_bytes(32),
    'provider': pk_provider,
}
write_request(subscriber_state)
status = lib.scal3_subscriber_register(buf)
response = read_response()
subscriber = response['subscriber']
verifier = response['verifier']

credential = {
    'verifier': verifier,
    'device': pk_device,
}
provider_state = credential | {
    'provider': pk_provider,
    'verifierSecret': ecdh(sk_provider, subscriber),
}
write_request(provider_state)
status = lib.scal3_provider_accept(buf)
response = read_response()

# Authentication

challenge_data = b"ts=1743930934&nonce=000001"
write_request({
    'randomness': hmac.digest(k_provider, challenge_data, hashlib.sha256)
})
status = lib.scal3_provider_challenge(buf)
response = read_response()
challenge = response['challenge']

client_data = b'{"operation":"log-in","session":"68c9eeeddfa5fb50"}'
client_data_hash = hashlib.sha256(client_data).digest()
write_request(subscriber_state | credential | {
    'subscriber': subscriber,
    'challenge': challenge,
    'hash': client_data_hash,
})
authentication = lib.scal3_subscriber_authenticate(buf)
response = read_response()
digest = response['digest']

r, s = ecdsa.sign(digest, sk_device, prehashed=True)
write_request({ 'proof': int_to_bytes(r, 32) + int_to_bytes(s, 32) })
status = lib.scal3_subscriber_pass(authentication, buf)
response = read_response()
sender = response['sender']
pass_attempt = response['pass']

write_request(provider_state | {
    'randomness': hmac.digest(k_provider, challenge_data, hashlib.sha256),
    'hash': client_data_hash,
    'passSecret': ecdh(sk_provider, sender),
    'pass': pass_attempt
})
status = lib.scal3_provider_prove(buf)
transcript = read_response()

write_request(credential | transcript | { 'hash': client_data_hash })
status = lib.scal3_verify(buf)
response = read_response()

assert status == 0
assert response['result'] == 'verified'

lib.scal3_buffer_free(buf)


0